# Disentangling cost drivers using cost change decomposition
This notebook demonstrates a Python library (`ccd`) for performing cost change decomposition, a method for separating and quantifying the contributions of different driving variables to cost change in a technology.  This method is described in [Kavlak G, McNerney J, Trancik JE, “Evaluating the causes of cost reduction in photovoltaic modules”, Energy Policy 123, 700 - 710 (2018)](https://www.sciencedirect.com/science/article/abs/pii/S0301421518305196).

Cost change decomposition is useful for studying a technology or production process for which cost be expressed as an equation in a set of underlying variables, and one has data on how these variables have changed (or will) over time.  For example, in the reference above, the authors analyzed the drivers of cost evolution in solar modules.  The costs of a solar module (dollars per kWh) can be modeled as
$$
C = \frac{\alpha}{\sigma A \eta y} \left[ A \nu \rho p_s + cA + p_0 \left( \frac{K}{K_0} \right)^{-b} \right],
$$
where symbols have the following meanings:

| Symbol   | Meaning                                | Type      |
|:--------:|----------------------------------------|-----------|
| $A$      | wafer area                             | variable  |
| $y$      | manufacturing yield                    | variable  |
| $\eta$   | module efficiency                      | variable  |
| $\eta$   | silicon usage                          | variable  |
| $p_s$    | polysilicon price                      | variable  |
| $c$      | per-area cost of non-silicon materials | variable  |
| $K$      | manufacturing plant size               | variable  |
| $\alpha$ | module area utilization                | parameter |
| $\sigma$ | solar constant                         | parameter |
| $\rho$   | wafer density                          | parameter |
| $K_0$    | reference manufacturing plant size     | parameter |
| $p_0$    | cost of reference plant                | parameter |
| $b$      | scaling factor                         | parameter |

Improvements in variables such as wafer area, manufacturing yield, and others caused significant reductions in the cost of solar modules over time.  Cost change decomposition provides a sytematic method for quantifying the separate contributions of these variable to cost reduction.  The mathematical details of this approach are given in [the reference above](https://www.sciencedirect.com/science/article/abs/pii/S0301421518305196).  The `ccd` python library encapsulates this method, giving users the ability to apply it more easily to their own cost problems.

The library exposes a simple interface to a user with **three main steps**:

1. **Specify the cost model**
2. **Bind the model to data**
3. **Compute cost change decompositions**

At each stage the user can inspect results to check that the problem parameters have been specified correctly.  The results are returned in a data frame from which a user can carry out further anaylsis.

The library is in a very early beta stage.  Beyond testing, future plans include providing additional ways to load data (e.g. csv), adding additional documentation, and providing a Matlab version of the code.

In [1]:
import numpy as np
import pandas as pd
from ccd import CostModel

## 1. Specify the cost model


In [8]:
# Initialize
cp = CostModel(
    title = 'A made up cost model.',
    equation = 'a1 * x1 x2+a2 * x3 *x4_cubed x2 * x1 + x5 + a3 a4 CF * x2'
)

# Rename cost components [optional]
cp.name_cost_components(['Materials','Labor','Equipment','O&M'])

# Specify which symbols are fixed parameters [optional]
cp.identify_parameters(['a1','a2','a3','a4'])

# Check that the equation was correctly parsed
print('\n\nCost model setup:')
print(cp)



Cost model setup:
Title:                A made up cost model.
Equation:             a1 x1 x2+a2 x3 x4_cubed x2 x1 + x5 + a3 a4 CF x2
Cost comp. names:     ['Materials', 'Labor', 'Equipment', 'O&M']
Cost components:      ['a1 x1 x2', 'a2 x3 x4_cubed x2 x1', 'x5', 'a3 a4 CF x2']
Symbols:              ['a1', 'x1', 'x2', 'a2', 'x3', 'x4_cubed', 'x5', 'a3', 'a4', 'CF']
Variables:            ['x1', 'x2', 'x3', 'x4_cubed', 'x5', 'CF']
Parameters:           ['a1', 'a2', 'a3', 'a4']
Num. cost components: 4
Num. symbols:         10
Num. variables:       6
Num. parameters:      4
Dependency matrix: 
[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 1.]]


## 2. Bind to data

In [3]:
# Enter data
time = [1980, 1985, 1993]
data = pd.DataFrame(index=time)
data['a1'] = 1 * np.ones((len(time), 1))
data['a2'] = 2.5 * np.ones((len(time), 1))
data['a3'] = 10 * np.ones((len(time), 1))
data['a4'] = 0.03 * np.ones((len(time), 1))
data['x1'] = [1, 1, 1.2]
data['x2'] = [100, 120, 150]
data['x3'] = [0.5, 0.6, 0.55]
data['x4_cubed'] = [5, 5.2, 5.7]
data['x5']= [10, 80, 120]
data['CF'] = [30, 40, 41]

# Bind data to symbols
cp.bind_data(data)

# Verify that data was bound correctly.  Examine costs in each period.
print('\n\nUpon binding the problem to variable data, cost components are automatically computed:')
cp.display_data()



Upon binding the problem to variable data, cost components are automatically computed:


,a1,a2,a3,a4,x1,x2,x3,x4_cubed,x5,CF,Materials,Labor,Equipment,O&M,Total_cost,Share_Materials,Share_Labor,Share_Equipment,Share_O&M
Time period,,,,,,,,,,,,,,,,,,,
1980,1.000,2.500,10.000,0.030,1.000,100,0.500,5.000,10,30,100.000,625.000,10,900.000,1635.000,0.061,0.382,0.006,0.550
1985,1.000,2.500,10.000,0.030,1.000,120,0.600,5.200,80,40,120.000,936.000,80,1440.000,2576.000,0.047,0.363,0.031,0.559
1993,1.000,2.500,10.000,0.030,1.200,150,0.550,5.700,120,41,180.000,1410.750,120,1845.000,3555.750,0.051,0.397,0.034,0.519


## 3. Compute cost change decompositions

In [4]:
# Specify desire time spans to study
time_spans = [
    (1980, 1985),
    (1985, 1993),
    (1980, 1993)
]

# Get change decomposition over these spans
cp.cost_change_decomposition(time_spans)

# Report results
print('\n\nAfter asking for change decompositions over particular time spans:')
cp.display_contributions()

# Show auxiliary quantities used in the computation [optional]
print('\n\nRepresentative values of cost components and log changes to variables:')
cp.display_change_data()



After asking for change decompositions over particular time spans:


,Total,Sum_of_changes(vars),Materials,Labor,Equipment,O&M,x1,x2,x3,x4_cubed,x5,CF
Time span,,,,,,,,,,,,
1980-1985,941.000,941.000,20.000,311.000,70,540.000,0.000,369.873,140.399,30.202,70.000,330.526
1985-1993,979.750,979.750,60.000,474.750,40,405.000,237.960,655.888,-100.689,106.239,40.000,40.351
1980-1993,1920.750,1920.750,80.000,785.750,110,945.000,200.782,980.293,91.988,126.461,110.000,411.226




Representative values of cost components and log changes to variables:


,Materials_rep,Labor_rep,Equipment_rep,O&M_rep,Dlog_a1,Dlog_x1,Dlog_x2,Dlog_a2,Dlog_x3,Dlog_x4_cubed,Dlog_x5,Dlog_a3,Dlog_a4,Dlog_CF
Time span,,,,,,,,,,,,,,
1980-1985,109.696,770.062,33.663,1148.927,0.000,0.000,0.182,0.000,0.182,0.039,2.079,0.000,0.000,0.288
1985-1993,147.978,1157.189,98.652,1634.144,0.000,0.182,0.223,0.000,-0.087,0.092,0.405,0.000,0.000,0.025
1980-1993,136.104,965.147,44.267,1316.450,0.000,0.182,0.405,0.000,0.095,0.131,2.485,0.000,0.000,0.312
